# Assignment 2: Jetty Scheduling

Name: **Moritz Grävinghoff, Frederic Weiss, Christian Husmann**\
Data of submission: **01.04.2024**

## Question 1

### Import all required packages

In [1]:
import pandas as pd
import numpy as np
from plotly import figure_factory as ff
import colorsys

### 1. Load data

In [2]:
def load_data(problem_nr):
    
    instance = pd.ExcelFile(f'../instances/inst{problem_nr}.xlsx')

    names = instance.parse(f'Problem_{problem_nr}', header=1, usecols='G').iloc[:,0].to_list()
    categories = instance.parse(f'Problem_{problem_nr}', header=1, usecols='H').iloc[:,0].to_list()
    arrivals = instance.parse(f'Problem_{problem_nr}', header=1, usecols='K').iloc[:,0].to_list()

    df_ships = pd.DataFrame({
        'name':names,
        'category':categories,
        'arrival':arrivals
    })

    df_unloading_time = instance.parse(f'Problem_{problem_nr}', header = 1, usecols = 'B:E').dropna()
    df_unloading_time = df_unloading_time.rename({'P1':'1','P2':'2','P3':'3','P4':'4'}, axis=1)
    df_unloading_time['category'] = df_unloading_time.index + 1

    return df_ships, df_unloading_time

### 2. Derive assigned jetty

In [3]:
def derive_assigned_jetty(df_ships, df_unloading_time):
    # each ship has to be unloaded at the jetty with the lowest unloading time

    df_ships = df_ships.merge(df_unloading_time, on='category')

    # unload at station with the lowest unloading time
    df_ships['assigned_station'] = df_ships[['1','2','3','4']].idxmin(axis=1).astype(int)
    df_ships['unloading_time'] = df_ships[['1','2','3','4']].min(axis=1)

    # all other times and category not needed anymore
    df_ships = df_ships.drop(['category','1','2','3','4'], axis=1)

    # Sort dataframe by arrival and then alphabetically
    df_ships = df_ships.sort_values(by=['arrival','name'], ignore_index=True)
    
    return df_ships

### 3. Schedule jetty

In [4]:
def schedule_jetty(df_ships):

    # add ships to queue
    queue = df_ships.index.tolist()

    # information on each station ((which) ship on it?) per period
    positions = [[None] * 4]

    # store number of finished ships, to know when scheduling is finished
    num_finished_ships = 0

    # period
    t = 0

    # add new ship if possible
    if (positions[t][0] == None) & (len(queue) > 0) & (df_ships.loc[queue[0],'arrival']-1<=t):
        # if position free and there is a ship in the queue, that already has arrived
        # then move ship to station 1
        positions[t][0] = queue.pop(0)

    while num_finished_ships < len(df_ships):

        # next period
        t += 1

        # calculate positions for the next period
        new_positions = [None] * 4

        # move ships in jetty
        for station, ship_in_station in reversed(list(enumerate(positions[t-1]))):
            if ship_in_station != None:
                # first, update unloading time after period t
                if (df_ships.loc[ship_in_station,'assigned_station']-1==station) and (df_ships.loc[ship_in_station,'unloading_time']>0):
                    # reduce unloading time by 1 as ship has been unloaded one period
                    df_ships.loc[ship_in_station,'unloading_time'] -= 1
                
                # now calculate positions for period t+1
                if df_ships.loc[ship_in_station,'unloading_time']>0:
                    # ship still has some unloading to do
                    if df_ships.loc[ship_in_station,'assigned_station']-1==station:
                        # ship is at assigned station in the next period as unloading is not finished yet
                        new_positions[station] = ship_in_station
                    elif df_ships.loc[ship_in_station,'assigned_station']-1>station:
                        # ship has not yet arrived at assigned station
                        if new_positions[station+1] == None:
                            # ship moves one station further if possible
                            new_positions[station+1] = ship_in_station
                        else:
                            # ship stays in same station otherwise
                            new_positions[station] = ship_in_station
                else:
                    # ship finished unloading and now just has to leave jetty
                    if station == 3:
                        # ship leaves jetty
                        new_positions[station] = None
                        num_finished_ships += 1
                    elif new_positions[station+1] == None:
                        # ship moves one station further
                        new_positions[station+1] = ship_in_station
                    else:
                        # ship stays in same station
                        new_positions[station] = ship_in_station
                
        # add new ship if possible
        if (new_positions[0] == None) and (len(queue) > 0) and (df_ships.loc[queue[0],'arrival']-1<=t):
            # if position free and there is a ship in the queue, that already has arrived
            # then move ship to station 1
            new_positions[0] = queue.pop(0)

        # add new positions to list if not yet finished
        if num_finished_ships < len(df_ships):
            positions.append(new_positions)
    
    return positions

### 4. Create gantt chart

#### Excursion

Extend the original plotly color list to be able to display 20 ships in instance 4. 

In [5]:
def generate_similar_colors(hex_colors, saturation_factor=.5, lightness_factor=.8):
    new_colors = []
    
    for hex_color in hex_colors:
        # Convert hex to RGB
        rgb_color = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (1, 3, 5))
        
        # Convert RGB to HLS
        h, l, s = colorsys.rgb_to_hls(*rgb_color)
        
        # Adjust saturation and lightness
        s *= saturation_factor
        l *= lightness_factor
        
        # Convert back to RGB
        new_rgb_color = tuple(round(c * 255) for c in colorsys.hls_to_rgb(h, l, s))
        
        # Convert RGB to hex
        new_hex_color = "#{:02x}{:02x}{:02x}".format(*new_rgb_color)
        
        new_colors.append(new_hex_color)
    
    return new_colors

# Original plotly list of colors
original_colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

extended_colors = original_colors + generate_similar_colors(original_colors)

In [6]:
def create_gantt_chart(schedule, df_ships, problem_nr):

    # dataframe to store information for gantt chart
    df_gantt = pd.DataFrame(columns=['Task','Start','Finish','Resource'])

    for station_idx in range(4):

        # for each station, extract the local schedule 
        station_schedule = np.array([period[station_idx] for period in schedule])

        for ship_idx, ship_row in df_ships.iterrows():

            # find periods for this ship-station combination
            periods_ship_station = np.argwhere(station_schedule==ship_idx).flatten()

            # add this interval to dataframe
            df_gantt = pd.concat([df_gantt, pd.DataFrame({'Task': f'Station {station_idx+1}', 'Start': [periods_ship_station.min()], 'Finish': [periods_ship_station.max()+1], 'Resource': ship_row['name']})], ignore_index=True)

    # create gantt chart
    fig = ff.create_gantt(df_gantt, index_col='Resource', show_colorbar=True, bar_width=0.4, group_tasks=True, title=f'Gantt Chart for Instance {problem_nr}; Finished after {len(schedule)} periods', colors=extended_colors)
    fig.update_layout(xaxis_type='linear', autosize=False, width=1000)

    return fig

### 5. Show results

In [7]:
for problem_nr in range(1,5):

    # (1) load data
    df_ships, df_unloading_time = load_data(problem_nr)

    # (2) derive assigned jetty
    df_ships = derive_assigned_jetty(df_ships, df_unloading_time)

    # (3) schedule jetty
    schedule = schedule_jetty(df_ships.copy())

    # (4) create gantt chart
    gantt = create_gantt_chart(schedule, df_ships, problem_nr)

    display(gantt)